# Instalar Telegram API

In [0]:
!pip install python-telegram-bot==5.3.0

# Configurar Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Seleccionar directorio de trabajo

In [0]:
%cd 'drive/My Drive/bot_utec'
!pwd
!ls

# Librerias

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import sys
import os
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import logging

print("Librerías importadas correctamente")

# Classify

In [0]:
def classify(image_path):
    # Read the image_data
    image_data = tf.io.gfile.GFile(image_path, 'rb').read()

    # Loads label file, strips off carriage return
    label_lines = [line.rstrip() for line in tf.io.gfile.GFile("tf_files/retrained_labels.txt")]

    # Unpersists graph from file
    with tf.io.gfile.GFile("tf_files/retrained_graph.pb", 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

    with tf.compat.v1.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        
        predictions = sess.run(softmax_tensor, \
                {'DecodeJpeg/contents:0': image_data})
        
        # Sort to show labels of first prediction in order of confidence
        top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
        
        result = None
        a=sorted(predictions[0], reverse=True)
        print(a)
        print(a[0])
        for node_id in top_k:
            human_string = label_lines[node_id]
            accuracy = predictions[0][node_id]
            
            print('%s (score = %.5f)' % (human_string, accuracy))
            if accuracy == a[0]:
              if accuracy>=0.3:
                result = human_string
                if result=="edificioa":
                  result="En este momento te encuentras cerca del edificio A,ahi podras hacer diferentes tramites escolares,ademas ahi esta rectoria"
                elif result=="edificiob":
                  result="En este momento te encuentras cerca del edificio B,de hecho estas muy cerca del edificio C y por supuesto de la entrada"
                elif result=="edificioc":
                  result="En este momento te encuentras cerca del edificio C,deberias entrar es un lugar genial"
                elif result=="edificiod":
                  result="En este momento te encuentras cerca del edificio D, tambien lo llaman aula modular, se ocupa a veces para talleres o cuando llegan a dar conferencias"
                elif result=="edificiof":
                  result="En este momento te encuentras cerca del edificio F, ahi se encuentran alumnos de mecatronica o carreras a fin"
                elif result=="edificiog":
                  result="En este momento te encuentras cerca del edificio G, ¡Genial!, ven visitanos, mis creadores se encuentran ahi,¡deberias saludarlos!"
                elif result=="edificioh":
                  result="En este momento te encuentras cerca del edificio H,aqui podras hacer diferentes movimientos relacionados con talleres, te interesa Radio, ¿porque no preguntas?"
                elif result=="edificioi":
                  result="En este momento te encuentras cerca del edificio I,ahi esta la carrera de salud reproductiva, ademas de que es la Biblioteca, pasa a leer un buen libro"
                elif result=="edificioj":
                  result="En este momento te encuentras cerca del edificio J,quizas estes buscando a alguien de enfermeria o terapia fisica porque ahi se encuentran, si quieres ir a rectoria, estas muy cerca"
                elif result=="estacionamientoj":
                  result="En este momento te encuentras cerca del estacionamiento que esta a un lado del Edificio J,ademas estas muy cerca de rectoria,cualquier duda visita el edificio A"
                elif result== "estacionamientocafeteria":
                  result="En este momento te encuentras cerca del estacionamiento que esta frente a la cafeteria,estas muy cerca de la entrada de la escuela,tambien del edificio B y del C"
                elif result=="explanada":
                  result="En este momento te encuentras enfrente del edificio G, en ese edificio estan mis creadores, ¡DEBERIAS VISITARLOS!, ademas a lado esta el area de hacer ejercicio, es muy tranquilo, porque no te sientas un rato"
                elif result=="areadeejercicio":
                  result="Ah mira esa es el area de ejercicio, pero muchos estudiantes solo la usan para pasar un momento tranquilo, a mi me agrada"
                elif result== "cafeteria":
                  result="¿tienes hambre?, esa es la cafeteria de la escuela, deberias visitarla"
                elif result=="canchas":
                  result="En este momento te encuentras cerca del estacionamiento que esta a un lado del Edificio J, eso que miras son las canchas, se usan para los talleres de futbol, o simplemente para pasar el rato"
                indicator=1
                break
              else:
                result= "Lo siento, a veces soy algo torpe, no reconozco muy bien tu foto, ¿serias tan amable de introducir otra?" 
    return result

print("Método para clasificar creado correctamente")

# Token de telegram

In [0]:
# ClassifyImagesBot
token = '' 

print("token registrado")

# Configuración del bot

In [0]:
# Enable logging
try:
    logging.basicConfig(
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
    logger = logging.getLogger(__name__)
except Exception as e:
    print("Error logging {}".format(e.args))

def start(bot, update):
    try:
        username = update.message.from_user.username
        message = "Hello " + username
        update.message.reply_text(message)
    except Exception as e:
        print("Error start {}".format(e.args))


def help(bot, update):
    try:
        username = update.message.from_user.username
        update.message.reply_text('Hello {}, please send a image for classify'.format(username))
    except Exception as e:
        print("Error help {}".format(e.args))

def analize(bot, update):
    try:
        message = "Receiving image..."
        update.message.reply_text(message)
        print(message)
        
        photo_file = bot.getFile(update.message.photo[-1].file_id)
        id_user = update.message.from_user.id
        id_file = photo_file.file_id
        id_analisis = str(id_user) + "-" + str(id_file)
        
        filename = os.path.join('downloads/', '{}.jpg'.format(id_analisis))
        photo_file.download(filename)
        message = "Image received, analyzing, please wait a few seconds"
        update.message.reply_text(message)
        print(message)
        
        result = classify(filename)
        print(result)
        update.message.reply_text(result)
        print("Waiting image..")
    except Exception as e:
        print("Error analize {}".format(e.args))

text_usuar=[]
text_bot=[]
def echo(bot, update):
    try:
        
        texto=update.message.text
        usuario=update.message.from_user['first_name']
        
        print(texto)

        if "hola" in texto or "HOLA" in texto or "Hola" in texto or "hOLA" in texto or "hola" in texto or "HOLA" in texto or "Hola" in texto or "hOLA" in texto:
          regreso="Hola ¿como estas?,¿necesitas algo?"
        elif texto=="Brenda" or texto=="BRENDA" or texto=="Brenda" or texto=="bRENDA" or "quien eres" in texto or "Quien eres" in texto:
          regreso="¡BRENDA!...esa soy yo, tu amiga y ayudante...Hola ¿como estas?,¿necesitas algo?"
        elif "como" in texto or "Como" in texto or "COMO" in texto or "cOMO" in texto:
          if "estas" in texto or "Estas" in texto or "eSTAS" in texto or "ESTAS" in texto:
            regreso="¿YO? AMM Excelente ¿que tal tu?"
          elif "llego" in texto or "Llego" in texto or "lLEGO" in texto or "LLEGO" in texto:
            regreso="ok, creo poder ayudarte, toma una foto de tu alrededor"
        elif "bien" in texto or "Bien" in texto or "BIEN" in texto or "bIEN" in texto:
          if "estoy" in texto or "me" in texto or "encuentro" in texto or "Estoy" in texto or "eSTOY" in texto or "ESTOY" in texto or "ME" in texto or "Me" in texto or "mE" in texto or "ENCUENTRO" in texto or "eNCUENTRO" in texto or "Encuentro" in texto:
            regreso="me alegro muchisimo de que te encuentres bien, dime como puedo ayudarte"
        elif "donde estoy" in texto or "Donde estoy" in texto or "donde" in texto or "Donde"  in texto or "dónde" in texto or "Dónde" in texto:
          regreso="amm porfavor toma una foto, necesito ver que hay a tu alrededor"
        elif "estoy" in texto or "ESTOY" in texto or "Estoy" in texto or "eSTOY" in texto:
          regreso="creo que podria a ayudarte a saber donde estas, toma una foto de tu alrededor"
        elif "ubicacion" in texto or "UBICACION" in texto or "uBICACION" in texto or "Ubicacion" in texto:
          regreso="ok puedo ayudarte, pero toma una foto de tu alrededor para que te diga donde estas"
        elif "necesito" in texto or "Necesito" in texto or "NECESITO" in texto or "nECESITO" in texto:
          if "ubicacion" in texto or "UBICACION" in texto or "uBICACION" in texto or "Ubicacion" in texto:
            regreso="ok puedo ayudarte, pero toma una foto de tu alrededor para que te diga donde estas"
          elif "encontrar" in texto or "eNCONTRAR" in texto or "Encontrar" in texto or "ENCONTRAR" in texto:
            regreso="ok puedo ayudarte a saber donde estas, pero toma una foto de tu alrededor para que te diga tu ubicacion"
          elif "donde" in texto or "Donde" in texto or "DONDE" in texto or "estoy" in texto or "ESTOY" in texto or "Estoy" in texto:
            regreso="ok puedo ayudarte a saber donde estas, pero toma una foto de tu alrededor para que te diga tu ubicacion" 
          else:
            regreso="ok creo comprender, bueno mi funcion es decirte donde estas asi que, puedes tomar una fotografia"
        elif "quiero" in texto or "Quiero" in texto or "QUIERO" in texto or "qUIERO" in texto:
            if "ir" in texto or "iR" in texto or "Ir" in texto  or "IR" in texto:
              if "edificioa" in texto or "edificioA" in texto or "edificio a" in texto or "edificio A" in texto or "EDIFICIOa " in texto or "EDIFICIOA" in texto or "EDIFICIO a" in texto or "EDIFICIO A" in texto or "Edificioa" in texto or "EdificioA" in texto or "Edificio a" in texto or "Edificio A" in texto:
                regreso="OK ,mmm dejame pensar, en este momento no estoy muy familiarizada pero mas adelante te ayudare"
        elif (texto=="claro" or texto=="Claro" or texto=="CLARO" or texto=="cLARO") and (text_bot[0]=="Hola ¿como estas?,¿necesitas algo?"):
          regreso="De acuerdo,dime ¿en que puedo ayudarte?"
        else:
            regreso="no entendi bien ,¿que necesitas?"


        
        update.message.reply_text(regreso)
        if(len(text_usuar)==0):
          text_usuar.append(texto)
          text_bot.append(regreso)   
        elif(len(text_usuar)==1):
          text_usuar.append(texto)
          text_usuar.pop(0)
          text_bot.append(regreso)
          text_bot.pop(0)
        print(text_usuar)
        print(text_bot)
        print(update.message.from_user['first_name'])
        print("Receiving text...")
        print("Waiting for another test...")
    except Exception as e:
        print("Error echo {}".format(e.args))

def error(bot, update, error):
    try:
        logger.warn('Update "%s" caused error "%s"' % (update, error))
    except Exception as e:
        print("Error error {}".format(e.args))

def main():
    try:
        print('ClassifyImagesBot init token')

        updater = Updater(token)
        # Get the dispatcher to register handlers
        dp = updater.dispatcher

        print('ClassifyImagesBot init dispatcher')


        # on different commands - answer in Telegram
        dp.add_handler(CommandHandler("start", start))
        dp.add_handler(CommandHandler("help", help))

        # on noncommand detect the document type on Telegram
        dp.add_handler(MessageHandler(Filters.text, echo))
        dp.add_handler(MessageHandler(Filters.photo, analize))

        # log all errors
        dp.add_error_handler(error)

        # Start the Bot
        updater.start_polling()
        print('ClassifyImagesBot ready')
        updater.idle()
    except Exception as e:
        print("Error main {}".format(e.message))

print("Bot configurado correctamente")


# Ejecutar el Bot

In [0]:
if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        print("Error name: {}".format(e.args))